In [0]:
# Load data
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)

In [0]:

# Create Managed Delta Table

events.write.format("delta").mode("overwrite").saveAsTable("events_table")

In [0]:
events.limit(10).display()

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-01T00:00:00.000Z,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2019-11-01T00:00:00.000Z,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2019-11-01T00:00:01.000Z,view,17302664,2053013553853497655,null,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
2019-11-01T00:00:01.000Z,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
2019-11-01T00:00:01.000Z,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
2019-11-01T00:00:01.000Z,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
2019-11-01T00:00:01.000Z,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
2019-11-01T00:00:02.000Z,view,15900065,2053013558190408249,null,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
2019-11-01T00:00:02.000Z,view,12708937,2053013553559896355,null,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24
2019-11-01T00:00:02.000Z,view,1004258,2053013555631882655,electronics.smartphone,apple,732.07,532647354,d2d3d2c6-631d-489e-9fb5-06f340b85be0


In [0]:
events = spark.table("events_table")
events.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
'''
Create Incremental Updates DataFrame
Simulated an incremental batch containing:

Existing event_time/user_session → UPDATE
New event_time/user_session → INSERT
'''

from pyspark.sql import Row
from pyspark.sql import functions as F

# Simulate incremental batch: updates and inserts
updates_data = [
    # Existing event_time/user_session → UPDATE
    Row("2026-01-10 10:00:00", "view", 101, 2001, "electronics.smartphone", "BrandA", 1200.50, 501, "session_1001"),
    Row("2026-01-10 10:05:00", "purchase", 102, 2002, "clothing.mens", "BrandB", 799.99, 502, "session_1002"),
    # New event_time/user_session → INSERT
    Row("2026-01-11 09:00:00", "cart", 103, 2003, "books.fiction", "BrandC", 89.99, 503, "session_999001"),
    Row("2026-01-11 09:10:00", "view", 104, 2004, "home.appliances", "BrandD", 1499.99, 504, "session_999002")
]

columns = [
    "event_time",
    "event_type",
    "product_id",
    "category_id",
    "category_code",
    "brand",
    "price",
    "user_id",
    "user_session"
]

updates_df = spark.createDataFrame(updates_data, columns) \
    .withColumn("event_time", F.to_timestamp("event_time"))

display(updates_df)
updates_df.printSchema()

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2026-01-10T10:00:00.000Z,view,101,2001,electronics.smartphone,BrandA,1200.5,501,session_1001
2026-01-10T10:05:00.000Z,purchase,102,2002,clothing.mens,BrandB,799.99,502,session_1002
2026-01-11T09:00:00.000Z,cart,103,2003,books.fiction,BrandC,89.99,503,session_999001
2026-01-11T09:10:00.000Z,view,104,2004,home.appliances,BrandD,1499.99,504,session_999002


root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
'''
Incremental MERGE (Upserts)
Used user_session and event_time as business keys.
'''

from delta.tables import DeltaTable

required_cols = {"event_time", "user_session"}
if not required_cols.issubset(set(updates_df.columns)):
    raise ValueError(f"updates_df must contain columns: {required_cols}. Found: {set(updates_df.columns)}")

deltaTable = DeltaTable.forName(spark, "events_table")

deltaTable.alias("t").merge(
    updates_df.alias("s"),
    "t.event_time = s.event_time AND t.user_session = s.user_session"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
# Check updates by filtering for updated user_session and event_time values
display(
    spark.table("events_table")
    .filter(
        "(user_session IN ('session_1001', 'session_1002', 'session_999001', 'session_999002')) AND " +
        "(event_time IN ('2026-01-10 10:00:00', '2026-01-10 10:05:00', '2026-01-11 09:00:00', '2026-01-11 09:10:00'))"
    )
)

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2026-01-10T10:00:00.000Z,view,101,2001,electronics.smartphone,BrandA,1200.5,501,session_1001
2026-01-10T10:05:00.000Z,purchase,102,2002,clothing.mens,BrandB,799.99,502,session_1002
2026-01-11T09:10:00.000Z,view,104,2004,home.appliances,BrandD,1499.99,504,session_999002
2026-01-11T09:00:00.000Z,cart,103,2003,books.fiction,BrandC,89.99,503,session_999001


In [0]:
%sql
-- Time Travel (Version History)
-- Checking Delta transaction log to identify when updates occurred.
DESCRIBE HISTORY events_table;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
9,2026-01-13T07:53:18.000Z,1055124393000177,lavy_kadali@yahoo.com,MERGE,"Map(predicate -> [""((event_time#15419 = event_time#15404) AND (user_session#15427 = user_session#15402))""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> true, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(934287304388572),0113-070124-zibm21ew-v2n,8,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 4, numTargetBytesAdded -> 9579, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 4127, materializeSourceTimeMs -> 147, numTargetRowsInserted -> 4, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 1460, numTargetRowsUpdated -> 0, numOutputRows -> 4, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 2401)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
8,2026-01-13T07:53:10.000Z,1055124393000177,lavy_kadali@yahoo.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(934287304388572),0113-070124-zibm21ew-v2n,7,WriteSerializable,false,"Map(numFiles -> 68, numRemovedFiles -> 68, numRemovedBytes -> 1188502790, numDeletionVectorsRemoved -> 0, numOutputRows -> 67501979, numOutputBytes -> 1188502790)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
7,2026-01-13T07:37:48.000Z,1055124393000177,lavy_kadali@yahoo.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(934287304388572),0113-070124-zibm21ew-v2n,6,WriteSerializable,false,"Map(numFiles -> 68, numRemovedFiles -> 11, numRemovedBytes -> 1212371621, numDeletionVectorsRemoved -> 0, numOutputRows -> 67501979, numOutputBytes -> 1188502790)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
6,2026-01-13T07:29:55.000Z,1055124393000177,lavy_kadali@yahoo.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(934287304388572),0113-070124-zibm21ew-v2n,5,WriteSerializable,false,"Map(numFiles -> 11, numRemovedFiles -> 23, numRemovedBytes -> 1202575271, numDeletionVectorsRemoved -> 0, numOutputRows -> 67501979, numOutputBytes -> 1212371621)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
5,2026-01-13T07:09:30.000Z,1055124393000177,lavy_kadali@yahoo.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(934287304388572),0113-070124-zibm21ew-v2n,4,WriteSerializable,false,"Map(numFiles -> 23, numRemovedFiles -> 22, numRemovedBytes -> 2576856722, numDeletionVectorsRemoved -> 0, numOutputRows -> 67501979, numOutputBytes -> 1202575271)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
4,2026-01-12T21:44:40.000Z,77907774700038,f9110c3c-95c5-42ee-aff2-370b927f4fe6,OPTIMIZE,"Map(predicate -> [], auto -> false, clusterBy -> [], zOrderBy -> [], batchId -> 0)","List(44588427254917, Predictive Optimization Job-a72b00f7-7f98-4043-bea3-5db82f886441, 203540330645589, 318488341157135, 77907774700038, manual)",null,0112-214340-a38let3l-v2n,3,SnapshotIsolation,false,"Map(numRemovedFiles -> 68, numRemovedBytes -> 11

In [0]:
# Time Travel: Viewing Record Before MERGE (Version 9) using SQL and schema
query = """
SELECT event_time, event_type, product_id, category_id, category_code, brand, price, user_id, user_session
FROM events_table VERSION AS OF 9
WHERE user_session IN ('session_1001', 'session_1002', 'session_999001', 'session_999002')
  AND event_time IN ('2026-01-10 10:00:00', '2026-01-10 10:05:00', '2026-01-11 09:00:00', '2026-01-11 09:10:00')
"""

display(spark.sql(query))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2026-01-10T10:00:00.000Z,view,101,2001,electronics.smartphone,BrandA,1200.5,501,session_1001
2026-01-10T10:05:00.000Z,purchase,102,2002,clothing.mens,BrandB,799.99,502,session_1002
2026-01-11T09:10:00.000Z,view,104,2004,home.appliances,BrandD,1499.99,504,session_999002
2026-01-11T09:00:00.000Z,cart,103,2003,books.fiction,BrandC,89.99,503,session_999001


In [0]:
%sql
-- OPTIMIZE & ZORDER (Performance)
-- Multiple small files → 1 optimized file
OPTIMIZE events_table
ZORDER BY (event_type, user_id);

path,metrics
,"List(26, 72, List(54997960, 88017762, 7.084546492307693E7, 26, 1841982088), List(2380, 19259650, 1.6507116236111112E7, 72, 1188512369), 0, List(minCubeSize(107374182400), List(0, 0), List(72, 1188512369), 0, List(72, 1188512369), 1, null), null, 0, 1, 72, 0, false, 0, 0, 1768292063061, 1768292093604, 8, 1, null, List(0, 0), null, 9, 9, 90317, 0, null)"


In [0]:
%sql
-- VACUUM (Storage Cleanup)
-- Removes unused old files while retaining time travel safety
VACUUM events_table;


path
""


In [0]:
%sql
-- ReCheck of Time Travel (Version History)
DESCRIBE HISTORY events_table;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
12,2026-01-13T08:16:15.000Z,1055124393000177,lavy_kadali@yahoo.com,VACUUM END,Map(status -> COMPLETED),null,List(934287304388572),0113-070124-zibm21ew-v2n,11,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
11,2026-01-13T08:16:14.000Z,1055124393000177,lavy_kadali@yahoo.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(934287304388572),0113-070124-zibm21ew-v2n,10,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
10,2026-01-13T08:14:54.000Z,1055124393000177,lavy_kadali@yahoo.com,OPTIMIZE,"Map(predicate -> [], auto -> false, clusterBy -> [], zOrderBy -> [""event_type"",""user_id""], batchId -> 0)",null,List(934287304388572),0113-070124-zibm21ew-v2n,9,SnapshotIsolation,false,"Map(numRemovedFiles -> 72, numRemovedBytes -> 1188512369, p25FileSize -> 63326430, numDeletionVectorsRemoved -> 0, minFileSize -> 54997960, numAddedFiles -> 26, maxFileSize -> 88017762, p75FileSize -> 78633877, p50FileSize -> 72017728, numAddedBytes -> 1841982088)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
9,2026-01-13T07:53:18.000Z,1055124393000177,lavy_kadali@yahoo.com,MERGE,"Map(predicate -> [""((event_time#15419 = event_time#15404) AND (user_session#15427 = user_session#15402))""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> true, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(934287304388572),0113-070124-zibm21ew-v2n,8,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 4, numTargetBytesAdded -> 9579, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 4127, materializeSourceTimeMs -> 147, numTargetRowsInserted -> 4, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 1460, numTargetRowsUpdated -> 0, numOutputRows -> 4, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 2401)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
8,2026-01-13T07:53:10.000Z,1055124393000177,lavy_kadali@yahoo.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(934287304388572),0113-070124-zibm21ew-v2n,7,WriteSerializable,false,"Map(numFiles -> 68, numRemovedFiles -> 68, numRemovedBytes -> 1188502790, numDeletionVectorsRemoved -> 0, numOutputRows -> 67501979, numOutputBytes -> 1188502790)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
7,2026-01-13T07:37:48.000Z,1055124393000177,lavy_kadali@yahoo.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(934287304388572),0113-070124-zibm21ew-v2n,6,WriteSerializable,false,"Map(numFiles -> 68, numRemovedFiles -> 11, numRemovedBytes -> 1212371621, numDeletionVectorsRemoved -> 0, numOutputRows -> 67501979, numOutputBytes -> 1188502790)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
6,2026-01-13T07:29:55.000Z,1055124393000177,lavy_kadali@yahoo.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(934287304388572),0113-070124-zibm21ew-v2n,5,WriteS